In [1]:
import os
import sys
import re
from pprint import pprint
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("C:/Users/99470/Downloads/data_cleaning.xlsx" , sheet_name = 'Usage',header = None)
df.columns = ['usage']
df['usage'] = df['usage'].str.lower().str.strip()
df.head()

,usage
0,3 pm mon 24th-mar-2014___0.384 kwh
1,5am 15-aug-2014___1.201 kwh
2,__8pm thu 20-mar-2014____1.523 kwh
3,6pm 23rd-apr-2014___0.424 kwh
4,_1am friday 19th-dec-2014___0.209 kwh


In [3]:
# cleaning data with regex
pattern = re.compile(r"(?:[\s\-\_]*)(\d+)(?:[\s\-\_]*)(am|pm)(?:[a-z\s]*)(\d{1,2})(?:.*)([a-z]{3})(?:[\s\-\_]*)(\d{4})(?:[\s\-\_]*)(\d+\.?\d+)")

def parse(row, pattern):
    text = row['usage']
    matched = pattern.match(text)
    return int(matched.group(1)), str(matched.group(2)),int(matched.group(3)), str(matched.group(4)), int(matched.group(5)), float(matched.group(6))

cl_df = df.apply(func = parse, args = (pattern,), axis = 1, result_type = 'expand')
cl_df.columns = ['hour', 'am_pm', 'day', 'month', 'year', 'kwh']

In [4]:
cl_df.head()

,hour,am_pm,day,month,year,kwh
0,3,pm,24,mar,2014,0.384
1,5,am,15,aug,2014,1.201
2,8,pm,20,mar,2014,1.523
3,6,pm,23,apr,2014,0.424
4,1,am,19,dec,2014,0.209


In [5]:
# creating datatime column

In [6]:
def convert_datetime(row):
    str_time = f"{row['year']}-{row['month']}-{row['day']}-{row['hour']}-{row['am_pm']}"
    return pd.to_datetime(datetime.strptime(str_time,"%Y-%b-%d-%I-%p"))

In [7]:
cl_df['datetime'] = cl_df.apply(func = convert_datetime, axis=1, result_type = 'expand')

In [8]:
cl_df

,hour,am_pm,day,month,year,kwh,datetime
0,3,pm,24,mar,2014,0.384,2014-03-24 15:00:00
1,5,am,15,aug,2014,1.201,2014-08-15 05:00:00
2,8,pm,20,mar,2014,1.523,2014-03-20 20:00:00
3,6,pm,23,apr,2014,0.424,2014-04-23 18:00:00
4,1,am,19,dec,2014,0.209,2014-12-19 01:00:00
...,...,...,...,...,...,...,...
8755,1,am,7,nov,2014,0.084,2014-11-07 01:00:00
8756,6,am,20,may,2014,1.027,2014-05-20 06:00:00
8757,2,am,8,apr,2014,0.052,2014-04-08 02:00:00
8758,9,pm,27,jan,2014,1.428,2014-01-27 21:00:00


In [9]:
# creating weekday column

In [10]:
day_of_week = {
    0:'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3:'Thursday',
    4:'Friday',
    5:'Saturday',
    6:'Sunday'
}

In [11]:
cl_df['weekday'] = cl_df['datetime'].dt.dayofweek.map(day_of_week)
cl_df.head()

,hour,am_pm,day,month,year,kwh,datetime,weekday
0,3,pm,24,mar,2014,0.384,2014-03-24 15:00:00,Monday
1,5,am,15,aug,2014,1.201,2014-08-15 05:00:00,Friday
2,8,pm,20,mar,2014,1.523,2014-03-20 20:00:00,Thursday
3,6,pm,23,apr,2014,0.424,2014-04-23 18:00:00,Wednesday
4,1,am,19,dec,2014,0.209,2014-12-19 01:00:00,Friday


In [12]:
# creating 24-hour column

In [13]:
cl_df['h24']= cl_df['datetime'].dt.hour
cl_df.head()

,hour,am_pm,day,month,year,kwh,datetime,weekday,h24
0,3,pm,24,mar,2014,0.384,2014-03-24 15:00:00,Monday,15
1,5,am,15,aug,2014,1.201,2014-08-15 05:00:00,Friday,5
2,8,pm,20,mar,2014,1.523,2014-03-20 20:00:00,Thursday,20
3,6,pm,23,apr,2014,0.424,2014-04-23 18:00:00,Wednesday,18
4,1,am,19,dec,2014,0.209,2014-12-19 01:00:00,Friday,1


In [14]:
# sorting data by datetime column

In [15]:
cl_df.sort_values(by = ['datetime'], ascending = True, inplace = True)
cl_df.head()

,hour,am_pm,day,month,year,kwh,datetime,weekday,h24
8425,12,am,1,jan,2014,0.684,2014-01-01 00:00:00,Wednesday,0
6316,1,am,1,jan,2014,0.340,2014-01-01 01:00:00,Wednesday,1
5068,2,am,1,jan,2014,0.332,2014-01-01 02:00:00,Wednesday,2
4106,3,am,1,jan,2014,0.396,2014-01-01 03:00:00,Wednesday,3
3239,4,am,1,jan,2014,0.388,2014-01-01 04:00:00,Wednesday,4


In [16]:
cl_df = cl_df.reset_index(drop = True)
cl_df.head()

,hour,am_pm,day,month,year,kwh,datetime,weekday,h24
0,12,am,1,jan,2014,0.684,2014-01-01 00:00:00,Wednesday,0
1,1,am,1,jan,2014,0.340,2014-01-01 01:00:00,Wednesday,1
2,2,am,1,jan,2014,0.332,2014-01-01 02:00:00,Wednesday,2
3,3,am,1,jan,2014,0.396,2014-01-01 03:00:00,Wednesday,3
4,4,am,1,jan,2014,0.388,2014-01-01 04:00:00,Wednesday,4


In [17]:
# Question 1: Answer-> B
print(np.round(cl_df["kwh"].mean(), 3))

0.782


In [18]:
# Question 2: Answer-> D
print(np.round(cl_df.loc[cl_df['month']=='feb',"kwh"].mean(), 3))

0.834


In [19]:
# Question 3: Answer-> A

In [20]:
print(cl_df.groupby(by= 'weekday').agg({'kwh':'mean'}).sort_values(by='kwh', ascending = False).head(1).index.values.item())

Sunday


In [21]:
print(np.around(cl_df['kwh'].rolling(4).sum().sort_values(ascending = False).head(1).values.item(), 3))

17.237


In [22]:
# Question 5: Answer-> B
monthly_flex_rate = {
    'jan':0.20,
    'feb':0.19,
    'mar':0.17,
    'apr':0.18,
    'may':0.22,
    'jun':0.27,
    'jul':0.24,
    'aug':0.19,
    'sep':0.18,
    'oct':0.15,
    'nov':0.14,
    'dec':0.19
}

In [23]:
month_kwh = cl_df.groupby(by=['month']).agg({'kwh':'sum'}).reset_index()

In [24]:
month_cost = month_kwh['month'].map(monthly_flex_rate) * month_kwh['kwh']

In [25]:
annual_cost_monthly = month_cost.sum()
print(np.round(annual_cost_monthly, 2))

1421.21


In [26]:
# Question 6: Answer-> C

In [27]:
no_flex = 0.21

In [28]:
hourly_flex = {
    '12AM until 1AM': 0.10,
    '1AM until 2AM': 0.10,
    '2AM until 3AM': 0.10,
    '3AM until 4AM': 0.10,
    '4AM until 5AM': 0.15,
    '5AM until 6AM': 0.20,
    '6AM until 7AM': 0.24,
    '7AM until 8AM': 0.24,
    '8AM until 9AM': 0.26,
    '9AM until 10AM': 0.20,
    '10AM until 11AM': 0.20,
    '11AM until 12PM': 0.26,
    '12PM until 1PM': 0.26,
    '1PM until 2PM': 0.20,
    '2PM until 3PM': 0.24,
    '3PM until 4PM': 0.18,
    '4PM until 5PM': 0.15,
    '5PM until 6PM': 0.30,
    '6PM until 7PM': 0.24,
    '7PM until 8PM': 0.24,
    '8PM until 9PM': 0.12,
    '9PM until 10PM': 0.11,
    '10PM until 11PM': 0.10,
    '11PM until 12AM': 0.10

}

In [29]:
# hourly flex cost

In [30]:
hour_kwh = cl_df.groupby(by = ['h24']).agg({'kwh':'sum'}).reset_index(drop=True).values.ravel()

In [31]:
annual_cost_hourly = np.sum(hour_kwh * np.array(list(hourly_flex.values())))

In [32]:
# no flex cost

In [33]:
annual_cost_no_flex = np.sum(cl_df['kwh']*no_flex)

In [34]:
annual_cost = [{'plan':'no flex', 'cost':np.around(annual_cost_no_flex,3)},
               {'plan':'hourly flex', 'cost':np.around(annual_cost_hourly,3)},
               {'plan':'monthly flex', 'cost':np.around(annual_cost_monthly,3)}
              ]
annual_cost

[{'plan': 'no flex', 'cost': 1438.104},
 {'plan': 'hourly flex', 'cost': 1368.976},
 {'plan': 'monthly flex', 'cost': 1421.206}]

In [35]:
sort = sorted(annual_cost, key = lambda x: x['cost'] , reverse = False)
pprint(annual_cost)
print(sort[0]['plan'])

[{'cost': 1438.104, 'plan': 'no flex'},
 {'cost': 1368.976, 'plan': 'hourly flex'},
 {'cost': 1421.206, 'plan': 'monthly flex'}]
hourly flex
